In [5]:
import sqlite3, statistics
from tqdm import tnrange, tqdm_notebook

conn = sqlite3.connect('../dataset/tile_placements.db')
c = conn.cursor()

In [6]:
def get_list_users(minimum_placements=20):
    c.execute("SELECT user_name FROM tile_placements GROUP BY user_name HAVING COUNT(*) >= {}".format(minimum_placements))
    result = list(c.fetchall())
    for x in range(len(result)):
        result[x] = result[x][0]
    result.remove('')
    return result

def get_user_placement_timestamp(user_name):
    c.execute("SELECT timestamp FROM tile_placements WHERE user_name = '{}';".format(user_name))
    result = list(c.fetchall())
    for x in range(len(result)):
        result[x] = result[x][0]
    return result

def get_timestamp_difference(timestamps):
    result = []
    timestamps.sort()
    for x in range(len(timestamps)-1):
        result.append(timestamps[x+1]-timestamps[x])
    return result

def rolling_window_std(timestamp_differences, window_size=5, threshold = 2): # 2 seconds
    result = [0]*len(timestamp_differences)
    for x in range(len(timestamp_differences)-window_size):
        window = timestamp_differences[x:x+window_size]
        std = statistics.stdev(window)
        if std <= threshold:
            for y in range(window_size):
                result[x+y] += 1
    return result

def insert_bot_results(user_name, actual_score, total_score, percentage):
    c.execute('''INSERT INTO bot_users (user_name, actual_score, total_score, percentage)
                 VALUES (?, ?, ?, ?);''', (user_name, actual_score, total_score, percentage))
    
def get_bots_higher_than(percentage = 50):
    c.execute('''SELECT COUNT(*) FROM bot_users WHERE percentage >= {};'''.format(percentage))
    print (c.fetchone())

In [7]:
user_names = get_list_users()

for user_name in tqdm_notebook(range(len(user_names))):
    user_name = user_names[user_name]
    user_timestamp_data = get_user_placement_timestamp(user_name)
    timestamp_difference = get_timestamp_difference(user_timestamp_data)
    rolling_window_std_results = rolling_window_std(timestamp_difference)
    total_score = 20+5*(len(rolling_window_std_results)-8)
    actual_score = sum(rolling_window_std_results)
    percentage = actual_score / total_score * 100
    insert_bot_results(user_name, actual_score, total_score, percentage)
conn.commit()

In [9]:
get_bots_higher_than(50)

(3125,)
